<a href="https://colab.research.google.com/github/leromn/-hayebonda-Latest-NetlifyVsCode-/blob/main/VideoCrafter_Image2Video_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#@title ⚙️ Install Dependencies
!pip install -q diffusers transformers accelerate safetensors
!pip install -q imageio imageio[ffmpeg] einops



In [5]:
#@title 🔄 Load the Model and Setup
import torch
from diffusers import StableVideoDiffusionPipeline
from PIL import Image
from google.colab import files

# Load the model
model_id = "stabilityai/stable-video-diffusion-img2vid-xt"
pipe = StableVideoDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    variant="fp16"
)
pipe.to("cuda")
pipe.enable_model_cpu_offload()
print("✅ Model loaded!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/3.05G [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

✅ Model loaded!


In [6]:
#@title 🖼️ Upload and Preprocess Image
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Load and resize image
image = Image.open(image_path).convert("RGB")
image = image.resize((1024, 576))
image.save("resized_input.png")
print("✅ Image uploaded and resized.")


Saving 7035fc41a5612d45db061870bcc4171e.jpg to 7035fc41a5612d45db061870bcc4171e.jpg
✅ Image uploaded and resized.


In [7]:
#@title 🎬 Generate the Video
from diffusers.utils import export_to_video

# Generate video frames
generator = torch.manual_seed(42)
frames = pipe(image, decode_chunk_size=8, generator=generator).frames[0]

# Export to MP4
video_path = "output_video.mp4"
export_to_video(frames, video_path, fps=6)
print("✅ Video generated and saved.")


  0%|          | 0/25 [00:00<?, ?it/s]

✅ Video generated and saved.


In [8]:
#@title 🎥 Display the Video
from IPython.display import HTML
from base64 import b64encode

# Encode video for HTML display
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

# Display the video
HTML(f"""
<video width=512 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")
